In [82]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd

In [83]:
def scraping_uf(uf: str):
    url_uf = f'https://www.ibge.gov.br/cidades-e-estados/{uf}.html'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    page = requests.get(url_uf, headers=headers)
    
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
        
        # Seleciona os indicadores da página
        indicadores = soup.select('.indicador')
        
        # Extrai os dados dos indicadores
        uf_dict = {
            dado.select('.ind-label')[0].text.strip(): dado.select('.ind-value')[0].text.strip()
            for dado in indicadores
            if dado.select('.ind-label') and dado.select('.ind-value')
        }
        
        return uf_dict
    else:
        print(f"Erro ao acessar {url_uf}: {page.status_code}")
        return None


    

In [84]:
def processar_dataframe(df):
    """
    Processa o DataFrame para:
    1. Remover datas [2023], [2022], etc., de todas as colunas, exceto "Capital".
    2. Ajustar a coluna "Capital" para manter apenas o ano e renomeá-la para "Ano".
    3. Remover o símbolo "R$" das colunas monetárias.
    """
    # Remover datas [2023], [2022], etc., em todas as colunas, exceto "Capital"
    df = df.applymap(lambda x: str(x).replace("[2023]", "").replace("[2022]", "").replace("[2021]", "").strip() if isinstance(x, str) else x)
    
    # Ajustar a coluna "Capital" para manter o ano entre colchetes e renomeá-la para "Ano"
    if "Capital" in df.columns:
        df["Capital"] = df["Capital"].str.replace(r".*\[(\d{4})\]$", r"\1", regex=True)
        df.rename(columns={"Capital": "Ano"}, inplace=True)
    
    # Remover o símbolo "R$" das colunas monetárias
    colunas_monetarias = [
        "Total de receitas brutas realizadas",
        "Total de despesas brutas empenhadas",
        "Rendimento mensal domiciliar per capita"
    ]
    for coluna in colunas_monetarias:
        if coluna in df.columns:
            df[coluna] = df[coluna].str.replace("R$ ", "", regex=False).str.strip()
    
    return df

In [85]:
# Lista de UFs
ufs = [
    "AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA",
    "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN",
    "RS", "RO", "RR", "SC", "SP", "SE", "TO"
]

# Coletar dados e armazenar em uma lista de dicionários
dados_completos = []
for uf in ufs:
    dados = scraping_uf(uf)
    if dados:
        dados["UF"] = uf  # Adiciona a UF como chave no dicionário
        dados_completos.append(dados)


In [86]:
# Cria um DataFrame com os dados
df = pd.DataFrame(dados_completos)
df_ajustado = processar_dataframe(df)
df_ajustado



C:\Users\marcell.oliveira\AppData\Local\Temp\ipykernel_41088\2214196043.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).replace("[2023]", "").replace("[2022]", "").replace("[2021]", "").strip() if isinstance(x, str) else x)


,Governador,Ano,Gentílico,Área Territorial,População residente,Densidade demográfica,Matrículas no ensino fundamental,IDH Índice de desenvolvimento humano,Total de receitas brutas realizadas,Total de despesas brutas empenhadas,Rendimento mensal domiciliar per capita,Total de veículos,UF
0,GLADSON DE LIMA CAMELI,2010,acriano,"164.173,525 km²",830.018 pessoas,"5,06 hab/km²",147.350 matrículas,"0,71","11.137.410.342,21 R$","10.302.403.420,85 R$",1.095 R$,350.273 veículos,AC
1,PAULO SURUAGY DO AMARAL DANTAS,2010,alagoano,"27.830,661 km²",3.127.683 pessoas,"112,38 hab/km²",439.014 matrículas,"0,684","20.927.817.222,51 R$","17.963.587.096,92 R$",1.110 R$,1.095.144 veículos,AL
2,CLÉCIO LUÍS VILHENA VIEIRA,2010,amapaense,"142.470,762 km²",733.759 pessoas,"5,15 hab/km²",131.948 matrículas,"0,688","11.249.572.238,02 R$","8.505.172.844,49 R$",1.520 R$,242.574 veículos,AP
3,WILSON MIRANDA LIMA,2010,amazonense,"1.559.256,365 km²",3.941.613 pessoas,"2,53 hab/km²",692.623 matrículas,"0,7","33.030.751.885,07 R$","30.029.370.828,75 R$",1.172 R$,1.130.055 veículos,AM
4,JERÔNIMO RODRIGUES SOUZA,2010,baiano,"564.760,429 km²",14.141.626 pessoas,"25,04 hab/km²",1.865.574 matrículas,"0,691","80.290.524.858,16 R$","77.868.037.416,52 R$",1.139 R$,5.120.353 veículos,BA
5,ELMANO DE FREITAS DA COSTA,2010,cearense,"148.894,447 km²",8.794.957 pessoas,"59,07 hab/km²",1.134.636 matrículas,"0,734","46.124.793.433,02 R$","37.423.640.356,93 R$",1.166 R$,3.753.826 veículos,CE
6,IBANEIS ROCHA BARROS JÚNIOR,2010,brasiliense,"5.760,783 km²",2.817.381 pessoas,"489,06 hab/km²",361.438 matrículas,"0,814","36.479.338.564,35 R$","33.897.143.557,68 R$",3.357 R$,2.083.081 veículos,DF
7,JOSÉ RENATO CASAGRANDE,2010,capixaba ou espírito-santense,"46.074,448 km²",3.833.712 pessoas,"83,21 hab/km²",498.357 matrículas,"0,771","34.291.544.903,51 R$","25.098.719.258,87 R$",1.915 R$,2.357.061 veículos,ES
8,RONALDO RAMOS CAIADO,2010,goiano,"340.242,860 km²",7.056.495 pessoas,"20,74 hab/km²",868.222 matrículas,"0,737","57.825.598.739,32 R$","39.420.594.475,73 R$",2.017 R$,4.726.950 veículos,GO
9,CARLOS ORLEANS BRANDÃO JÚNIOR,2010,maranhense,"329.651,495 km²",6.776.699 pessoas,"20,56 hab/km²",1.045.700 matrículas,"0,676","31.379.701.580,17 R$","24.496.014.978,83 R$",945 R$,2.132.527 veículos,MA


In [88]:
df_ajustado.to_csv('Extrai_web_scraping.csv')